# Documentação: Coleta de Issues do GitHub

## Introdução
Este script recupera informações sobre as issues de repositórios do GitHub de forma assíncrona. Ele filtra apenas as chaves relevantes e salva os dados processados em um arquivo CSV.

## Dependências
Antes de executar o script, instale as bibliotecas necessárias:

```bash
pip install aiohttp nest_asyncio pandas tqdm
```

## Estrutura do Script

### 1. Configuração do Loop Assíncrono
```python
nest_asyncio.apply()
```
Em ambientes como Jupyter Notebook, é necessário aplicar `nest_asyncio` para evitar conflitos com o loop de eventos assíncrono.

### 2. Definição das Chaves Relevantes
```python
chaves_relevantes = ["repo_id", "id", "number", "title", "user_id", "state", "comments", "created_at", "updated_at", "closed_at", "body", "reactions_total_count"]
```
Define quais informações serão extraídas das issues de repositórios do GitHub.

### 3. Função `flatten_issue`
```python
def flatten_issue(issue, parent_key=""):
```
Essa função achata um dicionário aninhado, transformando listas em strings separadas por vírgulas e mantendo apenas as chaves relevantes.

### 4. Função `fetch_issues_for_repo`
```python
async def fetch_issues_for_repo(session, repo_id, semaphore, timeout=1000):
```
Essa função busca as issues de um repositório no GitHub, tratando a paginação e aplicando a função `flatten_issue` para manter apenas as chaves relevantes.

### 5. Função `get_issues_data_async`
```python
async def get_issues_data_async(repo_ids, max_concurrent_requests=10):
```
Essa função gerencia a execução assíncrona para buscar informações das issues de múltiplos repositórios.

### 6. Função `get_issues_data`
```python
def get_issues_data(repo_ids):
```
Função wrapper para chamar a versão assíncrona de maneira síncrona.

### 7. Carregamento de IDs e Salvamento dos Dados
```python
repos_ids = pd.read_csv(f"{base_path}/repos_data.csv", encoding='utf-8')
repos_ids = repos_ids['id'].to_list()
```
Os IDs dos repositórios são lidos de um arquivo CSV, e os dados extraídos são salvos em um novo arquivo CSV no caminho definido por `base_path`.

**Fluxo Completo:**
1. Lê a lista de IDs de repositórios do GitHub a partir de um CSV.
2. Executa a busca de issues de forma assíncrona.
3. Filtra os dados mantendo apenas as chaves relevantes.
4. Salva os dados extraídos em um arquivo CSV:
   - `issues_data.csv`: Contém informações das issues coletadas.

### 8. Uso de Variáveis de Ambiente
O script requer duas variáveis de ambiente:
- `GITHUB_TOKEN`: Token de acesso à API do GitHub.
- `DIR_NAME`: Nome do diretório base para armazenamento dos dados processados.

Certifique-se de defini-las antes de executar o script.


# Main

In [1]:
import os
import pandas as pd
import asyncio
import aiohttp
from tqdm.asyncio import tqdm
import nest_asyncio

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
nest_asyncio.apply()

# Lista de chaves relevantes que desejamos coletar de cada issue.
chaves_relevantes = [
    "repo_id",           # Valor extraído do parâmetro da função
    "id",
    "number",
    "title",
    "user_id",
    "state",
    "comments",
    "created_at",
    "updated_at",
    "closed_at",
    "body",
    "reactions_total_count",
]

def flatten_issue(issue, parent_key=""):
    """
    Achata um dicionário de issue:
      - Se o valor for um dicionário, adiciona cada subchave como um novo item concatenando a chave pai e a subchave com underscore.
      - Se o valor for uma lista, converte a lista para uma string (itens separados por vírgula).
      - Apenas as chaves (ou chaves compostas para subchaves) que estiverem em 'chaves_relevantes' serão mantidas.
    
    Args:
        issue (dict): Dicionário original da issue.
        parent_key (str): Prefixo para as chaves (para dicionários aninhados).
    
    Returns:
        dict: Dicionário achatado contendo somente as chaves relevantes.
    """
    flattened = {}
    for key, value in issue.items():
        # Cria a chave com prefixo se houver parent_key
        new_key = f"{parent_key}_{key}" if parent_key else key

        if isinstance(value, dict):
            # Para cada subchave, cria a chave composta e verifica se ela é relevante.
            for sub_key, sub_value in value.items():
                composite_key = f"{new_key}_{sub_key}"
                if composite_key in chaves_relevantes:
                    flattened[composite_key] = sub_value
        elif isinstance(value, list):
            # Se a chave for relevante, converte a lista em uma string separada por vírgulas.
            if new_key in chaves_relevantes:
                flattened[new_key] = ", ".join(str(item) for item in value)
        else:
            if new_key in chaves_relevantes:
                flattened[new_key] = value
    return flattened

async def fetch_issues_for_repo(session, repo_id, semaphore, timeout=1000):
    """
    Busca os dados de issues de um repositório (identificado pelo seu ID),
    tratando a paginação e aplicando o flattening nos dados, mantendo somente as chaves relevantes.
    
    Args:
        session (aiohttp.ClientSession): Sessão HTTP.
        repo_id (int ou str): ID do repositório.
        semaphore (asyncio.Semaphore): Para limitar o número de requisições concorrentes.
        timeout (int): Timeout para a requisição.
        
    Returns:
        dict: Dicionário com o repo_id como chave e a lista de issues achatadas como valor.
    """
    # URL para coletar as issues a partir do ID do repositório.
    url = f"https://api.github.com/repositories/{repo_id}/issues"
    params = {'per_page': 100, 'page': 1}
    issues = []

    async with semaphore:
        while url:
            try:
                async with session.get(url, params=params, timeout=timeout) as response:
                    if response.status == 200:
                        page_data = await response.json()
                        issues.extend(page_data)
                        # Trata paginação: se houver link 'next', atualiza a URL.
                        if 'next' in response.links:
                            url = response.links['next']['url']
                            params = {}  # A URL já contém os parâmetros necessários
                        else:
                            url = None
                    else:
                        print(f"Erro na requisição para o repositório {repo_id}: {response.status}")
                        break
            except asyncio.TimeoutError:
                print(f"Timeout na requisição para o repositório {repo_id}")
                break

    flattened_issues = []
    for issue in issues:
        flat_issue = flatten_issue(issue)
        # Adiciona o repo_id à issue (caso não esteja presente)
        flat_issue["repo_id"] = repo_id
        flattened_issues.append(flat_issue)
    return {repo_id: flattened_issues}

async def get_issues_data_async(repo_ids, max_concurrent_requests=10):
    """
    Recupera de forma assíncrona os dados de issues para uma lista de repositórios,
    mantendo somente as chaves relevantes definidas em 'chaves_relevantes'.
    
    Args:
        repo_ids (list): Lista de IDs de repositórios.
        max_concurrent_requests (int): Número máximo de requisições concorrentes.
        
    Returns:
        dict: Dicionário mapeando cada repo_id à sua lista de issues achatadas.
    """
    token = os.getenv('GITHUB_TOKEN')
    if not token:
        raise ValueError("A variável de ambiente GITHUB_TOKEN não está definida.")
    
    headers = {
        'Authorization': f'Token {token}',
        'Accept': 'application/vnd.github.v3+json'
    }
    
    semaphore = asyncio.Semaphore(max_concurrent_requests)
    results = {}
    
    async with aiohttp.ClientSession(headers=headers) as session:
        tasks = [fetch_issues_for_repo(session, repo_id, semaphore) for repo_id in repo_ids]
        for coro in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Processando Repositórios"):
            repo_issue_data = await coro
            results.update(repo_issue_data)
    
    return results

def get_issues_data(repo_ids):
    """
    Função wrapper síncrona para recuperar os dados de issues para uma lista de repositórios.
    
    Args:
        repo_ids (list): Lista de IDs de repositórios.
    
    Returns:
        dict: Dicionário mapeando cada repo_id à sua lista de issues achatadas.
    """
    return asyncio.run(get_issues_data_async(repo_ids))


In [3]:
dir_name = os.getenv('DIR_NAME')
base_path = f'../data/{dir_name}'
os.makedirs(base_path, exist_ok=True)

repos_ids = pd.read_csv(f"{base_path}/repos_data.csv", encoding='utf-8')
repos_ids = repos_ids[repos_ids['open_issues_count'] > 0]['id'].to_list()

filename = f"{base_path}/issues_data.csv"
issues_data = get_issues_data(repos_ids) 


Processando Repositórios: 100%|██████████| 1001/1001 [00:48<00:00, 20.73it/s]


In [4]:
issues_list = []
for repo_id, issues in issues_data.items():
    for issue in issues:
        issues_list.append(issue)


issues_df = pd.DataFrame(issues_list)
issues_df.to_csv(filename, index=False, encoding='utf-8')

print(f'Gravados dados de {len(issues_df)} issues.')

Gravados dados de 5307 issues.
